In [7]:
import sys
import os

module_path = os.path.abspath(os.path.join('..'))
sys.path.append(os.path.dirname(module_path))
from collections import defaultdict
from rl.markov_process import FiniteMarkovProcess,FiniteMarkovRewardProcess,MarkovRewardProcess
from rl.distribution import Choose,Constant,Categorical
from dataclasses import dataclass

from typing import (Dict, Iterable, Generic, Sequence, Tuple,
                    Mapping, Optional, TypeVar, Callable)

In [26]:
from dataclasses import dataclass
import numpy as np
from rl.gen_utils.common_funcs import get_logistic_func, get_unit_sigmoid_func

@dataclass(frozen=True)
class StateMP3:
    num_up_moves: int
    num_down_moves: int


@dataclass
class StockPriceMP3(MarkovRewardProcess[StateMP3]):

    reward_func: Callable[[int,int], float]
    alpha3: float = 1.0  # strength of reverse-pull (non-negative value)
    
        
    def up_prob(self, state: StateMP3) -> float:
        total = state.num_up_moves + state.num_down_moves
        return get_unit_sigmoid_func(self.alpha3)(
            state.num_down_moves / total
        ) if total else 0.5

    def transition_reward(self, state: StateMP3) -> Categorical[StateMP3]:
        up_p = self.up_prob(state)

        return Categorical({
            (StateMP3(state.num_up_moves + 1, 
                      state.num_down_moves),self.reward_func(state.num_up_moves,1-state.num_down_moves)): up_p,
            (StateMP3(state.num_up_moves, state.num_down_moves + 1),self.reward_func(
            state.num_up_moves,state.num_down_moves-1)): 1 - up_p
        })


In [27]:
mrp=StockPriceMP3(reward_f)
def reward_f(p,q):
    return p+q

In [28]:
mrp.transition_reward(StateMP3(3,2))

{(StateMP3(num_up_moves=4, num_down_moves=2), 2): 0.4, (StateMP3(num_up_moves=3, num_down_moves=3), 4): 0.6}